# Smart Compliance

A fraud reduction strategy for compliance and risk management.

In [63]:
from deepface import DeepFace
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2

In [ ]:

import cv2
import pytesseract
from PIL import Image
import numpy as np

def preprocess_image_for_ocr(image_path):
    # Preprocess the image to improve OCR accuracy
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    preprocessed_image_path = '/mnt/data/preprocessed_image.jpg'
    cv2.imwrite(preprocessed_image_path, thresh)
    return preprocessed_image_path

def perform_ocr_on_image(image_path):
    # Perform OCR on the given image and return the extracted text
    preprocessed_image_path = preprocess_image_for_ocr(image_path)
    extracted_text = pytesseract.image_to_string(Image.open(preprocessed_image_path))
    return extracted_text.strip()


In [ ]:

import cv2
import numpy as np

def denoise_background(image_path, blur_intensity=5):
    # Load the image
    img = cv2.imread(image_path)

    # Apply Gaussian Blur to the image
    blurred_image = cv2.GaussianBlur(img, (blur_intensity, blur_intensity), 0)

    # Save the denoised image
    denoised_background_path = '/mnt/data/denoised_background.jpg'
    cv2.imwrite(denoised_background_path, blurred_image)

    return denoised_background_path

def denoise_id_and_face(image_path, diameter=9, sigmaColor=75, sigmaSpace=75):
    # Load the image
    img = cv2.imread(image_path)

    # Apply Bilateral Filter to the image
    denoised_image = cv2.bilateralFilter(img, diameter, sigmaColor, sigmaSpace)

    # Save the denoised image
    denoised_id_face_path = '/mnt/data/denoised_id_face.jpg'
    cv2.imwrite(denoised_id_face_path, denoised_image)

    return denoised_id_face_path


In [4]:
class FaceDetector():

    def __init__(self,faceCascadePath):
        self.faceCascade=cv2.CascadeClassifier(f"{cv2.data.haarcascades}{faceCascadePath}")


    def detect(self, image,
               scaleFactor=1.1,
               minNeighbors=5,
               minSize=(30,30)):

        #function return rectangle coordinates of faces for given image
        rects=self.faceCascade.detectMultiScale(image,
                                                scaleFactor=scaleFactor,
                                                minNeighbors=minNeighbors,
                                                minSize=minSize)
        return rects

## Selfie with passport/ID

Facial detection and recognition

In [ ]:
face_cascade_path = "haarcascade_frontalface_default.xml"
images = []
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        im = cv2.imread(os.path.join(dirname, filename))
        images.append(im)

for i, image in enumerate(images):
    # #image converted to grayscale
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    # #face detection performed
    faceRects=FaceDetector(faceCascadePath=face_cascade_path).detect(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))

    #faces drawn
    for j, (x,y,w,h) in enumerate(faceRects):
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.imwrite(f"./raw_data/detected_{i}_{j}.jpg",image[y:y+h, x:x+w])

    #image shown
    plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    plt.show()

## Photo passport/ID only

In [107]:
face_cascade_path = "haarcascade_frontalface_default.xml"

def detect_and_crop(folder, filename, image):
  gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  # #face detection performed
  rectangles=FaceDetector(faceCascadePath=face_cascade_path).detect(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30))

  #faces drawn
  for i, (x,y,w,h) in enumerate(rectangles):
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.imwrite(f"{folder}/{filename}_{i}.jpg",image[y:y+h, x:x+w])

In [109]:
if (os.path.exists("./raw_data/output") == False):
  os.mkdir("./raw_data/output")

for dirname, _, filenames in os.walk('./raw_data/comparison/'):
  for i, filename in enumerate(filenames):
    if filename != '.DS_Store':
      folder = f"./raw_data/output/{os.path.basename(dirname)}"

      if (os.path.exists(folder) == False):
        os.mkdir(folder)

      im = cv2.imread(os.path.join(dirname, filename))
      detect_and_crop(folder, i, im)

In [ ]:

DeepFace.verify("./raw_data/output/amit_passport_1/1_0.jpg", "./raw_data/output/amit_passport_1/2_1.jpg")

In [122]:
img1_path = ""
img2_path = ""

In [ ]:
models = [
  "VGG-Face",
  "Facenet",
  "Facenet512",
  "OpenFace",
  "DeepFace",
  "DeepID",
  "ArcFace",
  "Dlib",
  "SFace",
]

for model in models:
  result = DeepFace.verify(img1_path=img1_path,
      img2_path=img2_path,
      model_name = model
  )
  print(model, result)